In [ ]:
#HW3
#1. You are given a dataset having more variables than observations. Assuming that
    #there seems to be a linear relationship between the target variable and the input variables in the
    #dataset, why ordinary least squares (OLS) is a bad option to estimate the model parameters?
    #Which technique would be best to use? Why?
        #If the number of input variables is larger than the number of observations in the training dataset, then there is no longer a
        #unique least squares coecient estimates: the variance is infinite so the LS method can’t be
        #used at all. I would go with lasso, hoping to eliminate variabels that are adding complexity to the model but don't hold any 
        #significance when creating the actual models.
#2. For Ridge regression, if the regularization parameter, λ, is equal to 0, what are the
    #implications?
        #f a and c
            #a. Large coefficients in the linear model are not penalized.
            #c. The objective function is the same as ordinary least squares objective function.
#3. For Lasso Regression, if the regularization parameter, λ, is very high, which options are
    #true?
        #f. (a) and (b): Can be used to select important features of a dataset
            #and shrinks the coefficients of less important features to exactly 0.
#4. An important theoretical result of statistics and Machine Learning is the fact that model’s generalization error can be expressed as the sum of two very different errors:
        #•Bias: This part of the generalization error is due to wrong assumptions, such as assuming
        #that the data is linear when it is actually quadratic. A high-bias model is most likely to
        #under-fit the training data.
        #• Variance: This part is due to the model’s excessive sensitivity to small variations in the
        #training data. A model with many degrees of freedom (such as a high-degree polynomial
        #model) is likely to have high variance and thus overfit the training data.
            #Suppose you are using Ridge Regression and you notice that the training error and
            #the validation error are almost equal and fairly high. Would you say that the model suffers from
            #high bias or high variance? Should you increase the regularization parameter, λ, or reduce it?
                #I think the model is underfitting (high bias) and the data and it's best to reduce the lambda in order to get a better prediction on the data. 

In [31]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np


from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split



s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'CarPrice_Assignment.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
car_price = pd.read_csv(file_content_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [32]:
import warnings
warnings.simplefilter(action= 'ignore', category=FutureWarning)

In [33]:
#Creating Data Frame
df = list()

#Defining the input and target variables
x= car_price[['wheelbase', 'enginesize', 'horsepower', 'compressionratio', 'peakrpm', 'citympg', 'highwaympg']]
y= car_price['price']


#Repeating process 1000 times
for i in range(0, 1000):
    print(i)
    #Splitting data into train and test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2)
    
    #x_train = x_train.apply(l2_normalization, axis = 1)
    
    #Estimating Lambda
    lasso_cv= LassoCV(normalize= True, cv=5).fit(x_train, y_train)


    #Building lasso
    lasso_md= Lasso(alpha= lasso_cv.alpha_, normalize= True).fit(x_train, y_train)
    
    df.append(lasso_md.coef_)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [34]:
A = pd.DataFrame(df)
A

,0,1,2,3,4,5,6
0,239.363314,112.124366,39.442582,294.125447,2.454860,-107.571076,-0.00000
1,158.586334,113.293053,50.507569,260.670440,2.047145,-118.305476,-0.00000
2,184.489831,114.071338,43.559121,269.446033,1.939138,-107.838238,-0.00000
3,136.771385,127.754963,23.863724,302.695580,2.502227,-179.389860,-0.00000
4,157.033197,108.634214,33.918288,300.766940,2.014744,-176.496375,-0.00000
...,...,...,...,...,...,...,...
995,192.449759,102.994863,59.664510,256.453350,1.620850,-50.008594,-18.06021
996,219.371317,112.197766,35.672953,254.952172,2.001504,-84.656413,-42.38118
997,148.181880,87.774030,53.261490,399.151184,2.076937,-195.243165,-0.00000
998,120.246971,106.718590,50.897604,323.858379,1.884904,-157.438571,0.00000


In [35]:
print(sum(A[0] == 0.0))
print(sum(A[1] == 0.0))
print(sum(A[2] == 0.0))
print(sum(A[3] == 0.0))
print(sum(A[4] == 0.0))
print(sum(A[5] == 0.0))
print(sum(A[6] == 0.0))

0
0
0
0
0
18
774


In [36]:
#Dropping highwaympg
x_train= x_train.drop(columns= ['highwaympg'], axis=1)
x_test= x_test.drop(columns= ['highwaympg'], axis=1)

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2)

def l2_normalization(x):
    x_mean= np.mean(x)
    l2= np.sqrt(sum(x**2))
    return (x- x_mean) / l2

x_train= x_train.apply(l2_normalization, axis= 1)
x_test= x_test.apply(l2_normalization, axis= 1)

In [41]:
#Linear Regression
lm_md= LinearRegression().fit(x_train, y_train)

#Predictiong on test
lm_pred= lm_md.predict(x_test)

#Compute MSE of linear regression model
mse1= np.mean(np.power(y_test - lm_pred, 2))
print('MSE of this model is:', mse1)

MSE of this model is: 9591743.307648357


In [42]:
ridge= list()
for k in range(0,100):
    #Ridge Regression
    ridge_cv= RidgeCV(alphas= [.001, .01, .1, 1, 10, 100], cv=5).fit(x_train, y_train)

    #Extracting the best lambda
    cv_lambda= ridge_cv.alpha_
    #print('The best lambda of the ridge model is', cv_lambda)
    
    #storing lambda
    ridge.append(cv_lambda)   

import statistics 
from statistics import mode
print(mode(ridge))
    
#Building the ridge model
ridge_md= Ridge(alpha= .001).fit(x_train, y_train)

#predicting on test
ridge_pred= ridge_md.predict(x_test)

#Computing MSE
mse2= np.mean(np.power(y_test - ridge_pred,2 ))
print('MSE of Ridge model is:', mse2)

0.001
MSE of Ridge model is: 12399418.705710161


In [ ]:
#MSE of model 1: 9591743.307648357
#MSE of model 2: 12399418.705710161
#Based on looking at the MSE's of both models, we would want to use linear regression
#to estimate car prices because it has a smaller MSE (model 1)